In [11]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, IntSlider
from matplotlib.animation import FuncAnimation
from IPython.display import HTML


In [12]:
n = 4
# m = np.array([10,100,1000])
m = 1000
x = np.array([-1/2, -1/6, 1/6, 1/2])
y = np.array([1/4, 1/30, 1/30, 1/4])

In [13]:
## Helper functions

def relu(x):
    return np.maximum(0, x)

def indicator(condition):
    return condition.astype(int)

In [14]:
# 2-Lyaer ReLu Neural Network
def twoLayerReluNet(alpha_m, a, w, b, x):
    return 1/alpha_m * np.sum(a * relu(w * x + b))

In [15]:
## We use the following to fix the values of gamma, gamma_prime and alpha_m
gamma = 3/2
gamma_prime = -1/2

alpha_m = m**(gamma - gamma_prime)
kappa = m**(-gamma)
kappa_prime = m**(-gamma_prime)

beta_1m = math.sqrt(kappa*kappa_prime*alpha_m)
beta_2m = math.sqrt((kappa/kappa_prime)*alpha_m)

In [16]:
## Typical parameter gradients

# def gradient_flow(a,w,b):
#     grad_a = -(1/n) * np.sum([[(1/alpha_m) * relu((w*x[i])+b) * gradRiskwrtFunction(a,w,b,i)] for i in range(n)], axis=0).reshape(m)
#     grad_w = -(1/n) * np.sum([[(1/alpha_m) * a*x[i] * indicator(w*x[i]+b > 0) * gradRiskwrtFunction(a,w,b,i)] for i in range(n)], axis=0).reshape(m)
#     grad_b = -(1/n) * np.sum([[(1/alpha_m) * a * indicator(w*x[i]+b > 0) * gradRiskwrtFunction(a,w,b,i)] for i in range(n)], axis=0).reshape(m)
#     return grad_a, grad_w, grad_b

# def gradRiskwrtFunction(a,w,b,i):
#     return twoLayerReluNet(alpha_m, a, w, b, x[i]) - y[i]

def gradient_flow(a,w,b):
    # Compute wx_plus_b for all i and j
    wx_plus_b = np.outer(x,w) + b  # Shape: (n, m)

    # Apply ReLU activation
    relu_mask = (wx_plus_b > 0).astype(float)  # Shape: (n, m)
    relu = wx_plus_b * relu_mask  # Apply the mask

    # First term inside parentheses: (1 / alpha_m) * sum_j(a_j * ReLU_j)
    term2 = (np.sum(a * relu, axis=1) / alpha_m) - y  # Shape: (n,)

    # Full sum: sum relu multiplied with appropriate term2, over i
    full_sum_a = np.sum(relu * term2[:, np.newaxis], axis=0)  # Shape: (m,)

    # Multiply by aj * xi * 1{wx_plus_b > 0}
    term1_w = (a * x[:, np.newaxis] * relu_mask)  # Shape: (n, m)
    # Full sum: sum term1_w multiplied with appropriate term2, over i
    full_sum_w = np.sum(term1_w * term2[:, np.newaxis], axis=0)  # Shape: (m,)

    # Multiply by aj * 1{wx_plus_b > 0}
    term1_b = (a * relu_mask)  # Shape: (n, m)
    # Full sum: sum term1_b multiplied with appropriate term2, over i
    full_sum_b = np.sum(term1_b * term2[:, np.newaxis], axis=0)  # Shape: (m,)

    # Scale by constants
    grad_a = -full_sum_a / (n * alpha_m)  # Final shape: (m,)
    grad_w = -full_sum_w / (n * alpha_m)  # Final shape: (m,)
    grad_b = -full_sum_b / (n * alpha_m)  # Final shape: (m,)

    return grad_a, grad_w, grad_b

Normalised gradients

In [17]:
## Normalized parameter gradients

# def normalised_gradient_flow(a,w,b):
#     grad_a = - kappa_prime * (1/n) * np.sum([[kappa * relu((w*x[i])+b) * gradRiskwrtFunction(a,w,b,i)] for i in range(n)], axis=0).reshape(m)
#     grad_w = - kappa_prime * np.sum([[kappa * a*x[i] * indicator(w*x[i]+b > 0) * gradRiskwrtFunction(a,w,b,i)] for i in range(n)], axis=0).reshape(m)
#     grad_b = - kappa_prime * np.sum([[kappa * a * indicator(w*x[i]+b > 0) * gradRiskwrtFunction(a,w,b,i)] for i in range(n)], axis=0).reshape(m)
#     return grad_a, grad_w, grad_b

# def normalised_gradRiskwrtFunction(a,w,b,i):
#     return kappa * np.sum([[a[j] * relu((w[j]*x[i])+b[j]) - y[i]] for j in range(m)], axis=0)

# def normalise_param(a,w,b,t):
#     return a/beta_1m, w/beta_2m, b/beta_2m, t/(beta_1m * beta_2m)

Gradient Descent training

In [18]:
def gradient_descent(step_size, steps):
    
    a = np.random.normal(loc=0, scale=beta_1m, size=math.ceil(m))
    w = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m))
    b = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m))

    a_values = []
    w_values = []
    b_values = []

    #for n in range(math.ceil(time/step_size)):
    for n in range(steps): 
        grad_a, grad_w, grad_b = gradient_flow(a,w,b)
        a = a + step_size * grad_a
        w = w + step_size * grad_w
        b = b + step_size * grad_b

        # Save the current values of a, w, and b
        a_values.append(a.copy())
        w_values.append(w.copy())
        b_values.append(b.copy())

        print('Completed step  %d / %d' % (n, steps)) 

    return a_values, w_values, b_values

def gradient_descent_symmetric(step_size, steps):
    
    a = np.random.normal(loc=0, scale=beta_1m, size=math.ceil(m/2))
    w = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m/2))
    b = np.random.normal(loc=0, scale=beta_2m, size=math.ceil(m/2))

    a = np.concatenate((a, a))
    w = np.concatenate((w, -w))
    b = np.concatenate((b, b))

    a_values = []
    w_values = []
    b_values = []

    #for n in range(math.ceil(time/step_size)):
    for n in range(steps): 
        grad_a, grad_w, grad_b = gradient_flow(a,w,b)
        a = a + step_size * grad_a
        w = w + step_size * grad_w
        b = b + step_size * grad_b

        # Save the current values of a, w, and b
        a_values.append(a.copy())
        w_values.append(w.copy())
        b_values.append(b.copy())

        print('Completed step  %d / %d' % (n, steps)) 

    return a_values, w_values, b_values
        
# Assuming beta_1m, beta_2m, m, and gradient_flow are defined
n_steps = 200000
# a_values, w_values, b_values = gradient_descent_vector(4000, n_steps)

# # Convert lists to arrays for easier saving and plotting
# a_values = np.array(a_values)
# w_values = np.array(w_values)
# b_values = np.array(b_values)

# # Save the arrays to disk
# np.save('a_values_vector.npy', a_values)
# np.save('w_values_vector.npy', w_values)
# np.save('b_values_vector.npy', b_values)

Code Cell generating function development over time

In [19]:
# Load the arrays from the .npy files
a_values = np.load('weights\\a_values_vector_sym.npy')
w_values = np.load('weights\\w_values_vector_sym.npy')
b_values = np.load('weights\\b_values_vector_sym.npy')

# Define the range of x
x_graph = np.linspace(-0.5, 0.5, 800)
y_graph = np.zeros(len(x_graph))

# Define the function to plot the values at a given index
def plot_values(index):
    a = a_values[index]
    w = w_values[index]
    b = b_values[index]

    # Compute the corresponding y values
    for i in range(len(x_graph)):
        y_graph[i] = twoLayerReluNet(alpha_m,a,w,b,x_graph[i])

    # Plot the function
    plt.scatter(x, y, c='orange', alpha=0.5)
    plt.plot(x_graph, y_graph)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.xlim(-0.6, 0.6)
    plt.ylim(-0.01, 0.3)
    plt.show()

# Create an interactive slider
interact(plot_values, index=IntSlider(min=0, max=len(a_values)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='index', max=199999), Output()), _dom_classes=('widget-in…

<function __main__.plot_values(index)>

Code Cell generating min and max y values over time

In [ ]:
# Define the function to plot the values at a given index
def get_values(index):
    a = a_values[index]
    w = w_values[index]
    b = b_values[index]
    
    y_graph = np.zeros((int)(400))
    # Compute the corresponding y values
    for i in range(len(x_graph)):
        y_graph[i] = twoLayerReluNet(alpha_m,a,w,b,x_graph[i])

    return np.min(y_graph), np.max(y_graph)


x_graph = np.linspace(-0.5, 0.5, 400)
# Initialize an array to store y values
min_y_values = np.zeros((int)(n_steps/1000))
max_y_values = np.zeros((int)(n_steps/1000))
for n in range((int)(n_steps/1000)):
    min_y_values[n], max_y_values[n] = get_values(n*1000)

# Create an array of indices
indices = np.arange(n_steps/1000)

# Plot the min and max values
plt.figure(figsize=(10, 6))
plt.plot(indices, min_y_values, color='blue', label='Min y values')
plt.plot(indices, max_y_values, color='red', label='Max y values')
plt.xscale('log')
plt.xlabel('Index x1000')
plt.ylabel('y values')
plt.title('Min and Max y values at index')
plt.legend()
plt.grid(True)
plt.show()

Code cell for generating animation

In [ ]:
# # Assuming a_values, w_values, and b_values are already loaded
# a_values = np.load('weights_descent/a_values.npy')
# w_values = np.load('weights_descent/w_values.npy')
# b_values = np.load('weights_descent/b_values.npy')

# # Example x values (replace with your actual x values)
# x_graph = np.linspace(-0.5, 0.5, 100)
# y_graph = np.zeros(len(x_graph))


# # Create a figure and axis
# fig, ax = plt.subplots(figsize=(8, 6))
# line, = ax.plot([], [], lw=2)

# # Set limits for x and y axes
# ax.scatter(x, y, c='orange', alpha=0.5)
# ax.set_xlim(-0.6, 0.6)  # Set x-axis limits
# ax.set_ylim(-0.01, 0.26)  # Set y-axis limits
# ax.set_xlabel('x')
# ax.set_ylabel('y')
# ax.set_title('Animated Plot')
# ax.grid(True)

# # Initialization function to set up the background of each frame
# def init():
#     line.set_data([], [])
#     return line,

# # Animation function to update the plot
# def animate(i):
#     a = a_values[i*1000]
#     w = w_values[i*1000]
#     b = b_values[i*1000]
    
#     # Compute the corresponding y values
#     for j in range(len(x_graph)):
#         y_graph[j] = twoLayerReluNet(alpha_m,a,w,b,x_graph[j])
    
#     line.set_data(x_graph, y_graph)
#     return line,

# # Create the animation
# anim = FuncAnimation(fig, animate, init_func=init, frames=(int)(len(a_values)/1000), interval=1, blit=True, repeat=True, repeat_delay=100)

# # Save the animation
# anim.save('animations/animation.html', writer='html')
